In [5]:
import torch
import torch.nn as nn
from model import *
from data import *
from tools import *
from functions import *
from torch.utils.data import DataLoader


ISI Score Layer  1 :  tensor(0.5311, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  2 :  tensor(0.5003, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  3 :  tensor(0.4782, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  4 :  tensor(0.4684, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  5 :  tensor(0.4517, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  6 :  tensor(0.4496, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  7 :  tensor(0.4608, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  8 :  tensor(0.4672, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  9 :  tensor(0.4709, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  10 :  tensor(0.4733, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  11 :  tensor(0.4751, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  12 :  tensor(0.4765, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  13 :  tensor(0.4778, device='cuda:0', grad_f

In [2]:



## Model parameters

gamma_c = 1
gamma_w = 0.99
eps = 1e-12
nu = 0.5
zeta = 0.1 


# Hyperparameters

T = 100
K = 2
N = 3

lambda_1 = 0.04
lambda_2 = 0.25
rho_bounds_1 = [0.2,0.3]
rho_bounds_2 = [0.6,0.7]
rhos = [rho_bounds_1,rho_bounds_2]
lambdas = [lambda_1,lambda_2]

metaparameters_multiparam = get_metaparameters(rhos,lambdas)
metaparameters_titles_multiparam = ['Case A','Case B','Case C','Case D']


input_dim = N * N * K + K * K * N 
N_updates_W = 10
N_updates_C = 1
num_layers = 10
mode = 'test'
size = 1


data = MyDataset(T,K,N,metaparameters_multiparam,size)
dataloader = DataLoader(data, batch_size=1, shuffle=True)


In [3]:
(x,y) = next(iter(dataloader))

In [27]:
model = myModel(input_dim, N_updates_W, N_updates_C, num_layers=10, gamma_c=gamma_c, gamma_w=gamma_w, eps=eps, nu=nu, zeta=zeta).cuda()


In [28]:
Rx = cov_X(x[0])
Rx

tensor([[[[ 1.0000e+00, -1.9073e-08, -2.2650e-08],
          [-1.9073e-08,  1.0000e+00, -2.8610e-08],
          [-2.2650e-08, -2.8610e-08,  1.0000e+00]],

         [[-7.0360e-01, -5.8390e-02, -2.7983e-02],
          [ 4.5918e-02, -6.7165e-01,  6.6176e-02],
          [-5.3405e-02, -6.7522e-02, -4.8758e-01]]],


        [[[-7.0360e-01,  4.5918e-02, -5.3405e-02],
          [-5.8390e-02, -6.7165e-01, -6.7522e-02],
          [-2.7983e-02,  6.6176e-02, -4.8758e-01]],

         [[ 1.0000e+00, -1.7047e-07,  3.8147e-08],
          [-1.7047e-07,  1.0000e+00, -1.0282e-07],
          [ 3.8147e-08, -1.0282e-07,  1.0000e+00]]]], device='cuda:0')

In [29]:
W,C = initialize(N,K)

In [30]:
joint_isi(W,y[0])

tensor(0.4734, device='cuda:0')

In [45]:
model(Rx,W,C,y[0],mode)

(tensor([[[-0.5174, -1.2696],
          [ 0.9053, -0.1782],
          [-1.4097, -1.9092]],
 
         [[ 0.9695,  0.8883],
          [-1.8174,  0.2036],
          [-0.0957,  0.4201]],
 
         [[ 0.3256, -0.1550],
          [ 0.4584, -0.2224],
          [-0.7084,  0.8564]]], device='cuda:0'),
 tensor([[[0.9197, 0.9878, 0.9780],
          [0.3949, 0.5173, 0.5283]],
 
         [[0.3949, 0.5173, 0.5283],
          [1.0456, 1.0725, 1.0096]]], device='cuda:0'))

In [32]:
y[0]

tensor([[[ 0.1435, -0.7139],
         [ 1.0391, -1.0144],
         [ 1.0484,  0.2160]],

        [[ 2.1047,  1.7555],
         [-0.8142,  0.2095],
         [ 0.8352, -0.3957]],

        [[ 1.3956, -0.6532],
         [-1.1581, -4.0758],
         [ 1.0392, -3.9552]]], device='cuda:0')

In [33]:
W_predicted

tensor([[[-0.5174, -1.2696],
         [ 0.9053, -0.1782],
         [-1.4097, -1.9092]],

        [[ 0.9695,  0.8883],
         [-1.8174,  0.2036],
         [-0.0957,  0.4201]],

        [[ 0.3256, -0.1550],
         [ 0.4584, -0.2224],
         [-0.7084,  0.8564]]], device='cuda:0')

In [34]:
joint_isi(W_predicted,y[0])

tensor(0.4734, device='cuda:0')

In [29]:

for i, (X, A) in enumerate(dataloader):
    X = X[0].cuda()
    A = A[0].cuda()
    print("X.size : ", X.size())
    Rx = cov_X(X)
    W,C = initialize(N,K)

    model_5 = myModel(input_dim, N_updates_W, N_updates_C, num_layers=5, gamma_c=gamma_c, gamma_w=gamma_w, eps=eps, nu=nu, zeta=zeta).cuda()

    model_100 = myModel(input_dim, N_updates_W, N_updates_C, num_layers=100, gamma_c=gamma_c, gamma_w=gamma_w, eps=eps, nu=nu, zeta=zeta).cuda()


    W_predicted_5,_ = model_5(Rx,W,C,mode)
    W_predicted_100,_ = model_100(Rx,W,C,mode)

    print("ISI Score 5 layers: ", joint_isi(W_predicted_5,A))
    print("ISI Score 100 layers: ", joint_isi(W_predicted_100,A))    
    

X.size :  torch.Size([3, 100, 2])


TypeError: forward() takes 5 positional arguments but 6 were given

In [9]:
import torch


X = torch.load('X.pt')
A = torch.load('A.pt')
Winit = torch.load('Winit.pt')
Cinit = torch.load('Cinit.pt')

In [12]:


T = 100
K = 2
N = 3


input_dim = N * N * K + K * K * N 
N_updates_W = 10
N_updates_C = 1
num_layers = 170
mode = 'test'


model = myModel(input_dim, N_updates_W, N_updates_C, num_layers=num_layers, gamma_c=gamma_c, gamma_w=gamma_w, eps=eps, nu=nu, zeta=zeta).cuda()
Rx = cov_X(X)
W_predicted,_ = model(Rx,Winit,Cinit,A,mode)

ISI Score Layer  1 :  tensor(0.5473, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  2 :  tensor(0.5598, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  3 :  tensor(0.5721, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  4 :  tensor(0.5798, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  5 :  tensor(0.5766, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  6 :  tensor(0.5566, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  7 :  tensor(0.5203, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  8 :  tensor(0.4820, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  9 :  tensor(0.4471, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  10 :  tensor(0.4287, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  11 :  tensor(0.4184, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  12 :  tensor(0.4161, device='cuda:0', grad_fn=<DivBackward0>)
ISI Score Layer  13 :  tensor(0.4137, device='cuda:0', grad_f

In [13]:
W_predicted

tensor([[[ 3.1395e-02, -5.7970e-03],
         [-1.0061e+00, -1.0143e+00],
         [-1.3049e-01,  3.7982e-02]],

        [[-1.0340e+00, -1.0361e+00],
         [-5.5606e-02, -6.8842e-04],
         [ 1.8000e-01, -1.7650e-01]],

        [[-1.8344e-01, -1.7442e-01],
         [ 1.2523e-01,  3.9138e-02],
         [-1.0098e+00,  1.0184e+00]]], device='cuda:0',
       grad_fn=<PermuteBackward0>)

In [14]:
joint_isi(W_predicted,A)

tensor(0.3824, device='cuda:0', grad_fn=<DivBackward0>)

In [1]:
import torch
import torch.nn as nn

In [3]:
class ToyNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(2, 512)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

model = ToyNetwork()

In [4]:
nb_examples = 25
X = torch.randn(nb_examples,2)
X_alone = torch.randn(2)

In [5]:
X_alone

tensor([0.1541, 0.6563])

In [7]:
model(X_alone).shape

torch.Size([512])

In [8]:
model(X).shape

torch.Size([25, 512])